The source dataset consists of a set of audio and xml files describing them. It is necessary to generate data to verify proper operation of the future system for the detection of critical situations.

In [1]:
import numpy as np
import pandas as pd
from xml.dom import minidom

frames_big_df = pd.DataFrame(columns=['audio_num' ,'frame_num', 'neutral', 'glass', 'gunshot', 'scream'])
freq_rate = 32000
frameSize = 1024
hopSize = int(frameSize / 2)

big_window_size = 32

for xmlnum in range(1, 67):
    #load from xml
    x = minidom.parse("./XML/training/{0}.xml".format(str("%05d" % (xmlnum))))
    events = x.getElementsByTagName('events')[0]
    itemlist = events.getElementsByTagName('item')

    time_ranges = []
    for item in itemlist:
        startsecond = float(item.getElementsByTagName("STARTSECOND")[0].firstChild.nodeValue)
        endsecond = float(item.getElementsByTagName("ENDSECOND")[0].firstChild.nodeValue)
        classid = int(item.getElementsByTagName("CLASS_ID")[0].firstChild.nodeValue)
        time_ranges.append((startsecond, endsecond, classid))

    file_length = int(time_ranges[len(time_ranges) - 1][1]) + 1  #seconds

    #Determination accessories frame to the event
    #The frame length is 32 ms with 50% overlap
    frames_raw = []
    for i, start in enumerate(range(0, file_length * freq_rate, hopSize)):
        end = start + frameSize
        start_ms = start / freq_rate
        end_ms = end / freq_rate
        event = None
        for time_range in time_ranges:
            if time_range[0] < start_ms < time_range[1]:
                event = time_range[2]
                break
            elif time_range[0] < end_ms < time_range[1]:
                event = time_range[2]
                break
        frames_raw.append((i, event))

    #Formation of large windows with a duration of 32ms * 32 = 1,024s with 50% overlap 
    #If the event is more than half of the window, then the window contain this event
    
    #Events:
    #1 - neutral
    #2 - glass
    #3 - gunshot
    #4 - scream
    for frame_num, start_frame in enumerate(range(0, len(frames_raw), int(big_window_size / 2))):
        frame_big = frames_raw[start_frame:start_frame + big_window_size]
        count = np.zeros(4, dtype=np.int)
        for f in frame_big:
            if f[1] is None:
                count[0] += 1
            elif f[1] == 2:
                count[1] += 1
            elif f[1] == 3:
                count[2] += 1
            elif f[1] == 4:
                count[3] += 1
        max_classid = np.argmax(count)
        count_val = count[max_classid]
        max_classid += 1 #neutral is '1', not '0'
        if (count_val >= (int(big_window_size / 2))):
            frames_big_df.loc[len(frames_big_df)] = [xmlnum ,frame_num, (1 == max_classid), (2 == max_classid),
                                                    (3 == max_classid), (4 == max_classid)]
    print('audio ' + str(xmlnum) + ' is done')

audio 1 is done
audio 2 is done
audio 3 is done
audio 4 is done
audio 5 is done
audio 6 is done
audio 7 is done
audio 8 is done
audio 9 is done
audio 10 is done
audio 11 is done
audio 12 is done
audio 13 is done
audio 14 is done
audio 15 is done
audio 16 is done
audio 17 is done
audio 18 is done
audio 19 is done
audio 20 is done
audio 21 is done
audio 22 is done
audio 23 is done
audio 24 is done
audio 25 is done
audio 26 is done
audio 27 is done
audio 28 is done
audio 29 is done
audio 30 is done
audio 31 is done
audio 32 is done
audio 33 is done
audio 34 is done
audio 35 is done
audio 36 is done
audio 37 is done
audio 38 is done
audio 39 is done
audio 40 is done
audio 41 is done
audio 42 is done
audio 43 is done
audio 44 is done
audio 45 is done
audio 46 is done
audio 47 is done
audio 48 is done
audio 49 is done
audio 50 is done
audio 51 is done
audio 52 is done
audio 53 is done
audio 54 is done
audio 55 is done
audio 56 is done
audio 57 is done
audio 58 is done
audio 59 is done
audio 

In [2]:
frames_big_df [2000:2010]

,audio_num,frame_num,neutral,glass,gunshot,scream
2000,3,599,True,False,False,False
2001,3,600,True,False,False,False
2002,3,601,True,False,False,False
2003,3,602,True,False,False,False
2004,3,603,True,False,False,False
2005,3,604,True,False,False,False
2006,3,605,True,False,False,False
2007,3,606,True,False,False,False
2008,3,607,True,False,False,False
2009,3,608,True,False,False,False


In [3]:
frames_big_df.to_csv('./ml/trainset_w.csv', sep='\t')